# 0. Libraries

In [7]:
!pip install pandas -U

Requirement already up-to-date: pandas in /usr/local/lib/python3.6/dist-packages (1.1.3)


In [8]:
!pip install pandas_profiling -U

Requirement already up-to-date: pandas_profiling in /usr/local/lib/python3.6/dist-packages (2.9.0)


In [9]:
import pandas as pd
import numpy as np
import plotly.express as px
import pandas_profiling
from pandas_profiling import ProfileReport
from google.colab import drive
import warnings

In [10]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 1. Exploratory Analysis

In [42]:
ruta='/content/drive/My Drive/DS4A Team70/3. Exploratory Analysis/'
%cd '/content/drive/My Drive/DS4A Team70/3. Exploratory Analysis/'
%ls
df=pd.read_csv(ruta + 'SKIT_FINAL.csv',  sep=',' , encoding='latin-1')
df['Actividad']=df['Actividad'].str.lower()
st=pd.read_csv(ruta + 'Table_Stage.csv',  sep=';' , encoding='latin-1')
st['Actividad']=st['Actividad'].str.lower()

/content/drive/.shortcut-targets-by-id/1VIQ4q8_nY5C_dOyNoFA37-zusHZ2QmuP/DS4A Team70/3. Exploratory Analysis
'Copia de Exploration_Data.ipynb'    SKIT_FINAL.csv
 Etapas.gsheet                       Table_Stage.csv
 Exploration_Data.ipynb              Table_Stage.gsheet
'Revisión Base de Datos SKIT.gdoc'


The analysis that will be presented below includes general information of the database such as the total number of observations, type of variables, and the percentage of missing for each of the variables. The descriptive analysis is also performed for each of the variables, the number of categories of the categorical variables found, and the distribution of the data for each variable.






## 1.1 Initial DataBase

In [12]:
report_initial=ProfileReport(df, title="SKIT", explorative=True)
report_initial

Output hidden; open in https://colab.research.google.com to view.

In [13]:
#Se realiza el análisis univariado de las variables
descriptivo=df.describe()
descriptivo=descriptivo.transpose()
descriptivo=descriptivo.drop(columns=['25%', '50%', '75%'])
descriptivo.reset_index(inplace=True)

In [14]:
#Se calculan percentiles adicionales a los que genera el análisis predeterminado
per=df.quantile([0, .01, .05, .1, .25, .5, .75, .90, .95, .99, 1 ])
per=per.transpose()
per.reset_index(inplace=True)

In [15]:
univariado=descriptivo.merge(per, how="left", on=['index'])
univariado['variabilidad']=np.where(univariado[0.0]==univariado[1.0], '0%',
                                   np.where(univariado[0.05]==univariado[0.95], '10%',
                                           np.where(univariado[0.1]==univariado[0.9], '20%',
                                                   np.where(univariado[0.25]==univariado[0.75], '50%', 'variable'))))
univariado.head(15)

,index,count,mean,std,min,max,0.0,0.01,0.05,0.1,0.25,0.5,0.75,0.9,0.95,0.99,1.0,variabilidad
0,Importancia,70132.0,7.416671,1.074130,3.0,10.0,3.0,4.0,6.0,6.0,7.0,8.0,8.0,9.000000,9.000000,9.000,10.0,variable
1,Horas,70132.0,3.530956,2.362573,1.0,18.0,1.0,1.0,1.0,1.0,2.0,3.0,4.0,8.000000,8.000000,8.000,18.0,variable
2,Horas planeadas,70132.0,127.015599,554.247415,0.0,4855.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,160.000000,720.000000,2600.000,4855.0,50%
3,Porcentaje,70132.0,0.000891,0.004461,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000824,0.004533,0.025,0.1,50%
4,Año,70132.0,2016.164290,2.372196,2012.0,2020.0,2012.0,2012.0,2013.0,2013.0,2014.0,2016.0,2018.0,2019.000000,2020.000000,2020.000,2020.0,variable
5,Mes,70132.0,6.316589,3.364213,1.0,12.0,1.0,1.0,1.0,2.0,3.0,6.0,9.0,11.000000,12.000000,12.000,12.0,variable
6,Día,70132.0,15.724035,8.741832,1.0,31.0,1.0,1.0,2.0,4.0,8.0,16.0,23.0,28.000000,29.000000,31.000,31.0,variable


In [16]:
cols = list(df.columns)[12:]
cols.insert(0,"Proyecto")
aux = df[cols]
aux = pd.melt(aux, id_vars = ['Proyecto'], value_vars = list(df.columns)[12:], var_name = 'Etapa', value_name = 'Proporcion') 

In [17]:
fig = px.box(aux[1:], x = "Etapa", y = "Proporcion")
fig.show()

Output hidden; open in https://colab.research.google.com to view.

The analysis of the original base showed that there are a total of 70,132 observations and 15 variables, however of those 15 variables, 5 variables are duplicated information (Year, Month, Day, Person in charge, Percentage), 2 are key variables (client name, project) and we have 8 variables for analysis. 
One of the most important findings of the analysis is the high concentration in 0's of the variable "planned hours", when reviewing this data with the entity, we found that this 0 means that this information is not available, then it could not be used as an element for the construction of the objective variable of the project, but it should be made a proposal with the executed hours.

## 1.2 Build New Variables 

Taking into account that the database is a log of the different activities carried out in the project, it is considered necessary to transform the database at the level (client name, project) and build variables that provide us with more information on the particular behavior of the projects and more easily identify outliers or relevant information. 

In [18]:
## DataBase Transformation

df['Fecha'] = pd.to_datetime(df['Fecha'])
df['Actividad']=np.where(df['Actividad'].isin(['soporte correctivo horario normal', 'soporte de negocio', 'soporte nocturno', 'soporte fin de semana']),'soporte',df['Actividad'])
df1=df.merge(st, how="left", on=['Actividad'])

## Unique Stage
aux_st= df1[["Nombre Cliente", "Proyecto", "Etapa"]].groupby(["Nombre Cliente", "Proyecto"]).nunique()['Etapa'].reset_index()

## Table Proportion hours per activity/ total hours 
aux = df[~df['Actividad'].isin(['tareas administrativas y financieras', 'recursos humanos', 'gerencia de proyectos'])][["Nombre Cliente", "Proyecto", "Actividad", "Horas"]].groupby(["Nombre Cliente", "Proyecto", "Actividad"]).sum().reset_index()
aux = aux.sort_values(by=['Nombre Cliente', 'Proyecto', 'Horas'], ascending=False)

aux1 = df[~df['Actividad'].isin(['tareas administrativas y financieras', 'recursos humanos', 'gerencia de proyectos'])][["Nombre Cliente", "Proyecto", "Horas"]].groupby(["Nombre Cliente", "Proyecto"]).sum().reset_index()
aux_f = aux.merge(aux1, how="left", on=["Nombre Cliente", "Proyecto"])
aux_f['prop_horasproy_horasact'] = aux_f['Horas_x']/aux_f['Horas_y']

aux_h = aux_f[['Nombre Cliente', 'Proyecto', 'Horas_y']].drop_duplicates()
aux_h.rename(columns={"Horas_y": "Horas Ejecutadas Propias Proyecto"}, inplace=True)

aux_def=aux_f.pivot(index=['Nombre Cliente','Proyecto'], columns='Actividad', values='prop_horasproy_horasact')
aux_def=aux_def.reset_index()
aux_def=aux_def.fillna(0)

aux_fun = df[~df['Actividad'].isin(['tareas administrativas y financieras', 'recursos humanos', 'gerencia de proyectos'])][["Nombre Cliente", "Proyecto", "Actividad", "Funcionario"]].groupby(["Nombre Cliente", "Proyecto", "Actividad"]).nunique().reset_index()
aux_fun = aux_fun.sort_values(by=['Nombre Cliente', 'Proyecto'], ascending=False)
aux_def_fun=aux_fun.pivot(index=['Nombre Cliente','Proyecto'], columns='Actividad', values='Funcionario')
aux_def_fun=aux_def_fun.reset_index()
aux_def_fun.columns = ['Nombre Cliente','Proyecto','capacitación_funcionarios','capacitación interna_funcionarios','consultoría_funcionarios','desarrollo_funcionarios','garantía_funcionarios','infraestructura skit_funcionarios','instalaciones_funcionarios','investigación_funcionarios','mercadeo_funcionarios','post venta_funcionarios','preventa_funcionarios','pruebas_funcionarios','reprocesos_funcionarios','requerimientos_funcionarios','seguimiento ofertas koncilia_funcionarios','soporte_funcionarios','venta_funcionarios']
aux_def_fun=aux_def_fun.fillna(0)


In [19]:

t1=df.groupby(['Nombre Cliente','Proyecto']).agg({
    'Fecha':[max, min],
    'Horas':[sum],
    'Actividad':['nunique'],
    'Importancia':[max, min],
    'Funcionario':['nunique'],
    'Versión':['nunique']
})
t1.columns=t1.columns.droplevel(1)
t1=t1.reset_index()
t1.columns=['Nombre Cliente', 'Proyecto', 'Fecha Maxima', 'Fecha Minima', 'Horas Ejecutadas Totales', 'Numero de Actividades', 'Max_importancia', 'Min_Importancia', 'Num_Funcionarios', 'Num_Versiones']

## Project Months 
t1['Dias_Proyecto'] = t1['Fecha Maxima']- t1['Fecha Minima']
t1['Dias_Proyecto'] = [x.days for x in t1.Dias_Proyecto]
t1['Meses_Proyecto'] = t1['Fecha Maxima']- t1['Fecha Minima']
t1['Meses_Proyecto'] = [x.days//30 for x in t1.Meses_Proyecto]

## Consolidation others variables
tf=t1.merge(aux_st, how="left", on=['Nombre Cliente', 'Proyecto'])
tf=tf.merge(aux_h, how="left", on=['Nombre Cliente', 'Proyecto'])
tf=tf.merge(aux_def, how="left", on=["Nombre Cliente", "Proyecto"])
tf=tf.merge(aux_def_fun, how="left", on=["Nombre Cliente", "Proyecto"])
tf.head()

,Nombre Cliente,Proyecto,Fecha Maxima,Fecha Minima,Horas Ejecutadas Totales,Numero de Actividades,Max_importancia,Min_Importancia,Num_Funcionarios,Num_Versiones,Dias_Proyecto,Meses_Proyecto,Etapa,Horas Ejecutadas Propias Proyecto,capacitación,capacitación interna,consultoría,desarrollo,garantía,infraestructura skit,instalaciones,investigación,mercadeo,post venta,preventa,pruebas,reprocesos,requerimientos,seguimiento ofertas koncilia,soporte,venta,capacitación_funcionarios,capacitación interna_funcionarios,consultoría_funcionarios,desarrollo_funcionarios,garantía_funcionarios,infraestructura skit_funcionarios,instalaciones_funcionarios,investigación_funcionarios,mercadeo_funcionarios,post venta_funcionarios,preventa_funcionarios,pruebas_funcionarios,reprocesos_funcionarios,requerimientos_funcionarios,seguimiento ofertas koncilia_funcionarios,soporte_funcionarios,venta_funcionarios
0,Cliente 1,Proyecto 133,2013-11-14,2013-11-14,2,1,8,8,1,1,0,0,1,2.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.00000,0.000000,0.000000,0.0,1.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Cliente 10,Proyecto 20,2016-01-26,2013-06-11,37,2,8,7,3,1,959,31,2,37.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.00000,0.000000,0.000000,0.0,0.513514,0.486486,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0
2,Cliente 11,Proyecto 5,2013-09-26,2013-09-18,7,1,8,8,1,1,8,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Cliente 12,Proyecto 127,2020-09-11,2013-02-01,6124,14,10,4,21,1,2779,92,6,5927.0,0.083179,0.073562,0.0,0.159946,0.0,0.006749,0.00135,0.030538,0.152354,0.0,0.345200,0.067319,0.0,0.064788,0.010461,0.002700,0.001856,8.0,10.0,0.0,11.0,0.0,6.0,2.0,5.0,1.0,0.0,10.0,7.0,0.0,9.0,1.0,3.0,3.0
4,Cliente 12,Proyecto 130,2016-11-29,2013-08-05,1296,9,9,4,7,2,1212,40,6,1210.0,0.006612,0.016529,0.0,0.741322,0.0,0.000000,0.00000,0.014050,0.000000,0.0,0.008264,0.048760,0.0,0.090909,0.000000,0.073554,0.000000,1.0,1.0,0.0,5.0,0.0,0.0,0.0,1.0,0.0,0.0,3.0,3.0,0.0,3.0,0.0,1.0,0.0


## 1.3 Report Transformated Data

In [20]:
report_initial=ProfileReport(tf, title='SKIT TRANSFORMATION', explorative=True)
report_initial

KeyboardInterrupt: ignored

In [ ]:
report_initial.to_file("report_base_transformada.html")

In [21]:
#Se realiza el análisis univariado de las variables
descriptivo=tf.describe()
descriptivo=descriptivo.transpose()
descriptivo=descriptivo.drop(columns=['25%', '50%', '75%'])
descriptivo.reset_index(inplace=True)

In [22]:
#Se calculan percentiles adicionales a los que genera el análisis predeterminado
per=tf.quantile([0, .01, .05, .1, .25, .5, .75, .90, .95, .99, 1 ])
per=per.transpose()
per.reset_index(inplace=True)

In [23]:
univariado=descriptivo.merge(per, how="left", on=['index'])
univariado['variabilidad']=np.where(univariado[0.0]==univariado[1.0], '0%',
                                   np.where(univariado[0.05]==univariado[0.95], '10%',
                                           np.where(univariado[0.1]==univariado[0.9], '20%',
                                                   np.where(univariado[0.25]==univariado[0.75], '50%', 'variable'))))
univariado.head(15)

,index,count,mean,std,min,max,0.0,0.01,0.05,0.1,0.25,0.5,0.75,0.9,0.95,0.99,1.0,variabilidad
0,Horas Ejecutadas Totales,163.0,1519.220859,3083.110130,2.0,24987.000000,2.0,2.0,4.0,8.0,37.0,309.000000,1729.500000,3728.600000,5447.400000,15960.540000,24987.000000,variable
1,Numero de Actividades,163.0,5.926380,3.806360,1.0,16.000000,1.0,1.0,1.0,1.0,2.0,6.000000,9.000000,11.000000,12.000000,13.380000,16.000000,variable
2,Max_importancia,163.0,8.478528,0.705686,6.0,10.000000,6.0,6.0,8.0,8.0,8.0,9.000000,9.000000,9.000000,9.000000,9.000000,10.000000,variable
3,Min_Importancia,163.0,5.343558,1.533093,3.0,8.000000,3.0,3.0,3.0,4.0,4.0,5.000000,6.000000,8.000000,8.000000,8.000000,8.000000,variable
4,Num_Funcionarios,163.0,7.245399,7.606115,1.0,57.000000,1.0,1.0,1.0,1.0,2.0,5.000000,9.000000,17.000000,20.000000,30.040000,57.000000,variable
5,Num_Versiones,163.0,4.496933,6.768093,1.0,44.000000,1.0,1.0,1.0,1.0,1.0,2.000000,5.000000,11.800000,19.900000,32.420000,44.000000,variable
6,Dias_Proyecto,163.0,744.969325,784.398767,0.0,2899.000000,0.0,0.0,0.0,2.2,75.0,434.000000,1206.500000,1928.600000,2399.700000,2888.320000,2899.000000,variable
7,Meses_Proyecto,163.0,24.423313,26.072997,0.0,96.000000,0.0,0.0,0.0,0.0,2.0,14.000000,40.000000,63.800000,79.500000,96.000000,96.000000,variable
8,Etapa,163.0,3.858896,1.999621,1.0,7.000000,1.0,1.0,1.0,1.0,2.0,4.000000,5.500000,6.800000,7.000000,7.000000,7.000000,variable
9,Horas Ejecutadas Propias Proyecto,159.0,1474.119497,3002.667711,1.0,24435.000000,1.0,2.0,4.0,7.8,36.5,291.000000,1623.000000,3588.400000,5399.200000,15838.060000,24435.000000,variable


After analyzing the variables, we found that the activities that have a greater participation, with respect to the variable hours executed in the projects, are Desarrollo (Development) with 30% and Requerimientos (Requierements) with 24.5%, followed by Preventas (Pre-sales) with 14%, Pruebas (Tests) with 11% and Gerencia de Proyectos (Project Management) with 7.6%. The other activities have very little participation within the projects.

Moreover, we find that the projects have a high concentration in 1, 5 and 6 stages, however, it is important to validate with the entity whether those projects with one stage are completed or not, since we are evaluating whether restrictions on compliance with stages that the entity manages to verify if there is success or not in the projects.

In [25]:
plot = ["Etapa", "Numero de Actividades", "Num_Funcionarios", "Num_Versiones", "Meses_Proyecto"]
for i in plot:
  fig = px.box(tf, y = i)
  fig.show()

In [26]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Box(y=tf['capacitación'], name='capacitación',
                marker_color = 'indianred'))
fig.add_trace(go.Box(y=tf['capacitación interna'], name = 'capacitación interna',
                marker_color = 'lightseagreen'))
fig.add_trace(go.Box(y=tf['consultoría'], name = 'consultoría',
                marker_color = 'indianred'))
fig.add_trace(go.Box(y=tf['desarrollo'], name = 'desarrollo',
                marker_color = 'lightseagreen'))
fig.add_trace(go.Box(y=tf['garantía'], name = 'garantía',
                marker_color = 'indianred'))
fig.add_trace(go.Box(y=tf['infraestructura skit'], name = 'infraestructura skit',
                marker_color = 'lightseagreen'))
fig.add_trace(go.Box(y=tf['instalaciones'], name = 'instalaciones',
                marker_color = 'indianred'))
fig.add_trace(go.Box(y=tf['investigación'], name = 'investigación',
                marker_color = 'lightseagreen'))
fig.add_trace(go.Box(y=tf['mercadeo'], name = 'mercadeo',
                marker_color = 'indianred'))
fig.add_trace(go.Box(y=tf['post venta'], name = 'post venta',
                marker_color = 'lightseagreen'))
fig.add_trace(go.Box(y=tf['preventa'], name = 'preventa',
                marker_color = 'indianred'))
fig.add_trace(go.Box(y=tf['pruebas'], name = 'pruebas',
                marker_color = 'lightseagreen'))
fig.add_trace(go.Box(y=tf['reprocesos'], name = 'reprocesos',
                marker_color = 'indianred'))
fig.add_trace(go.Box(y=tf['requerimientos'], name = 'requerimientos',
                marker_color = 'lightseagreen'))
fig.add_trace(go.Box(y=tf['seguimiento ofertas koncilia'], name = 'seguimiento ofertas koncilia',
                marker_color = 'indianred'))
fig.add_trace(go.Box(y=tf['soporte'], name = 'soporte',
                marker_color = 'lightseagreen'))
fig.add_trace(go.Box(y=tf['venta'], name = 'venta',
                marker_color = 'indianred'))
fig.show()

In [28]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Box(y=tf['capacitación_funcionarios'], name='capacitación_funcionarios',
                marker_color = 'indianred'))
fig.add_trace(go.Box(y=tf['capacitación interna_funcionarios'], name = 'capacitación interna_funcionarios',
                marker_color = 'lightseagreen'))
fig.add_trace(go.Box(y=tf['consultoría_funcionarios'], name = 'consultoría_funcionarios',
                marker_color = 'indianred'))
fig.add_trace(go.Box(y=tf['desarrollo_funcionarios'], name = 'desarrollo_funcionarios',
                marker_color = 'lightseagreen'))
fig.add_trace(go.Box(y=tf['garantía_funcionarios'], name = 'garantía_funcionarios',
                marker_color = 'indianred'))
fig.add_trace(go.Box(y=tf['infraestructura skit_funcionarios'], name = 'infraestructura skit_funcionarios',
                marker_color = 'lightseagreen'))
fig.add_trace(go.Box(y=tf['instalaciones_funcionarios'], name = 'instalaciones_funcionarios',
                marker_color = 'indianred'))
fig.add_trace(go.Box(y=tf['investigación_funcionarios'], name = 'investigación_funcionarios',
                marker_color = 'lightseagreen'))
fig.add_trace(go.Box(y=tf['mercadeo_funcionarios'], name = 'mercadeo_funcionarios',
                marker_color = 'indianred'))
fig.add_trace(go.Box(y=tf['post venta_funcionarios'], name = 'post venta_funcionarios',
                marker_color = 'lightseagreen'))
fig.add_trace(go.Box(y=tf['preventa_funcionarios'], name = 'preventa_funcionarios',
                marker_color = 'indianred'))
fig.add_trace(go.Box(y=tf['pruebas_funcionarios'], name = 'pruebas_funcionarios',
                marker_color = 'lightseagreen'))
fig.add_trace(go.Box(y=tf['reprocesos_funcionarios'], name = 'reprocesos_funcionarios',
                marker_color = 'indianred'))
fig.add_trace(go.Box(y=tf['requerimientos_funcionarios'], name = 'requerimientos_funcionarios',
                marker_color = 'lightseagreen'))
fig.add_trace(go.Box(y=tf['seguimiento ofertas koncilia_funcionarios'], name = 'seguimiento ofertas koncilia_funcionarios',
                marker_color = 'indianred'))
fig.add_trace(go.Box(y=tf['soporte_funcionarios'], name = 'soporte_funcionarios',
                marker_color = 'lightseagreen'))
fig.add_trace(go.Box(y=tf['venta_funcionarios'], name = 'venta_funcionarios',
                marker_color = 'indianred'))
fig.show()

Grafica util, demasiada informacion para visualizar

Si se quita la etiqueta de color se podria mostrar informacion de los meses donde mas se trabaja.

*Sugerencia: una variable que podriamos considerar es el mes en que se recibe el proyecto y la cantidad de proyectos en simultaneo que se tienen para ese momento.

In [29]:
#Grafica util, demasiada informacion para visualizar
#Si se quita la etiqueta de color se podria mostrar informacion de los meses donde mas se trabaja.
#*Sugerencia: una variable que podriamos considerar es el mes en que se recibe el proyecto y la cantidad de proyectos en simultaneo que se tienen para ese momento.

aux = df.groupby(["Proyecto", df["Fecha"].dt.year, df["Fecha"].dt.month])["Actividad"].count().to_frame()#.plot(kind="bar", title  = t)
aux.index.set_names(["Proyecto", "Año", "Mes"], inplace=True)
aux.reset_index(inplace = True)
aux['Periodo'] = pd.to_datetime(aux[['Año','Mes']].astype(str).agg('-'.join, axis = 1))
fig = px.bar(aux, x = 'Periodo', y = 'Actividad')#, color = "Proyecto")
fig.show()

In [30]:
# Misma grafica discriminada por proyectos
proyectos = df.Proyecto.unique()
for p in proyectos:
  aux = df[df.Proyecto == p]
  aux = aux.groupby([aux["Fecha"].dt.year, aux["Fecha"].dt.month])["Actividad"].count().to_frame()#.plot(kind="bar", title  = t)
  aux.index.set_names(["Año", "Mes"], inplace=True)
  aux.reset_index(inplace = True)
  aux['Periodo'] = pd.to_datetime(aux[['Año','Mes']].astype(str).agg('-'.join, axis = 1))
  fig = px.bar(aux, x = 'Periodo', y = 'Actividad')
  fig.show()
  break

In [31]:
tf['Etapa']=tf['Etapa'].astype(int)
for i in range(8):
  fig=px.scatter(tf , x=tf.iloc[:, 8+i], y='Horas Ejecutadas Propias Proyecto', color='Etapa', log_y=True, log_x=True ,labels={
                     "x": tf.columns[8+i]})
  fig.update_layout(title= tf.columns[8+i] + ' vs Horas Ejecutadas Propias Proyecto'   )
  fig.show()


En el gráfico se puede ver que la mayoria de proyectos fueron ejecutados entre 5 y 7 etapas, tambien es posible visualizar que hay algunos datos atípicos en la cantidad de horas ejecudadas para algunos proyectos. 

In [32]:
fig = px.box(tf, x="Etapa", y="Horas Ejecutadas Propias Proyecto" , points='all')
fig.show()

In [43]:
df.to_hdf('data.h5', key='original', mode='w')
tf.to_hdf('data.h5', key='new_variables')
univariado.to_hdf('data.h5', key='univariado')

/usr/local/lib/python3.6/dist-packages/pandas/io/pytables.py:270: PerformanceWarning:


your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->axis0] [items->None]


/usr/local/lib/python3.6/dist-packages/pandas/io/pytables.py:270: PerformanceWarning:


your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_items] [items->None]


